In [7]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
#from sklearn import cluster
from kneed import KneeLocator
import os

In [11]:
#read the pkl file
current_dir = os.getcwd()  # gets current working directory
project_dir = os.path.dirname(current_dir)  
directory_path = os.path.dirname(current_dir) + '/data'

################# PATH FOR JONAS ################# directory_path = os.getcwd() + '\\data'

df = pd.read_pickle(directory_path + '/full_movies.pkl')
expat_library= df[df['tags2'].str.contains('Australia|australia|australian|Australian') | df['description'].str.contains('Australia|australia|australian|Australian') | df['description2'].str.contains('Australia|australia|australian|Australian') | df['title'].str.contains('Australia|australia|australian|Australian')
]

In [35]:
df.shape

(409, 17)

In [10]:
#read json file activity
activity = pd.read_json(directory_path + '/activities.json')

In [43]:
activity.head()
activity.groupby('user_id').count()

,content_id,activity,datetime
user_id,,,
1,100,100,100
2,108,108,108
3,101,101,101
4,83,83,83
5,108,108,108


## TF-IDF + kmeans clustering

In [38]:
#tf-idf and kmeans for all movies
# Convert the text data to a matrix of TF-IDF features
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df['description'])


inertia = []
#calculate the optimal number of clusters
K = range(1, 20)
for k in K:
    kmeanModel = KMeans(n_clusters=k,random_state=0)
    kmeanModel.fit(tfidf)
    inertia.append(kmeanModel.inertia_)


kn = KneeLocator(K, inertia, curve='convex', direction='decreasing')
print(kn.knee)


13


In [39]:
# Cluster the documents using KMeans
num_clusters = 13
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf)

# Assign cluster labels to each document
clusters = km.labels_.tolist()
df['cluster'] = clusters

In [40]:
# Print the top 10 terms for each cluster
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names()
for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :13]:
        print(' %s' % terms[ind], end='')
    print()



Cluster 0: young book julia based donaldson axel scheffler friendship animated 2009 world dragon children
Cluster 1: artist portrait sarah narrated film intimate artistic new australian memories walks terence priscilla
Cluster 2: story disability science people greatest international extraordinary australian geography filmed live attempt fashion
Cluster 3: time beach stranded summer school lost cadets neville recalls deserted kangaroo gave flashback
Cluster 4: performed ballet music orchestra australian opera tale australia victoria love symphony costumes tells
Cluster 5: home fielding spectacular strong culture paquita commonwealth forests female ancient vast queen cold
Cluster 6: heart new christmas people adventure warming story celebrate year old jaydyn best love
Cluster 7: family 2012 father man life stick love friends 2010 years couple pina forced
Cluster 8: gospel music singer mark manchester semi lisser gorton monastery final talented 2021 hosts
Cluster 9: journey girl working 

/Users/ekaterinamazur/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [41]:
joined = pd.merge(df, activity, left_on='id', right_on='content_id', how='right')
#make from list in tag2 column to string and delete [] in the string
joined['tags2'] = joined['tags2'].apply(lambda x: str(x).replace('[', '').replace(']', ''))
#drop NaN values in id column
joined.dropna(subset=['id'], inplace=True)
#check if tag2 column or description, description2, title columns consist of 'australia' or 'Australia'
expat_data = joined[joined['tags2'].str.contains('Australia|australia|australian|Australian') | joined['description'].str.contains('Australia|australia|australian|Australian') | joined['description2'].str.contains('Australia|australia|australian|Australian') | joined['title'].str.contains('Australia|australia|australian|Australian')
]
#dataframe (movies) about Australia
expat_data.shape

(72, 22)

In [49]:
#top-3 clusters for each user in joined dataframe
top3 = joined.groupby('user_id')['cluster'].apply(lambda x: x.value_counts().nlargest(3).index.tolist()).reset_index(name='top3')



In [50]:
top3

,user_id,top3
0,1,"[6.0, 0.0, 2.0]"
1,2,"[4.0, 0.0, 2.0]"
2,3,"[7.0, 9.0, 10.0]"
3,4,"[0.0, 9.0, 7.0]"
4,5,"[0.0, 7.0, 6.0]"


In [ ]:
#suggest movies for each user from top-3 clusters